# Basic skill template for broker

This will provide a basic showcase skill for the broker to handle requests from clients. It will return a random number between 0 and 10.

In [1]:
import socketio
import time
import random
import multiprocessing as mp
import logging

In [2]:
# Definition of the skill and url to the broker
url = "http://127.0.0.1:4852"
#url = "http://care.ukp.informatik.tu-darmstadt.de:4853"  # our dev broker

skill_definition = {
    "name": "test_skill",
    "description": "This is a test skill",
    "version": "0.0.1",
    "author": "test_author",
    "input": {
        "data": {
            "text": {
                "type": "string",
                "description": "This text will be ignored"
            }
        },
        "example": {
            "text": "This is a test text"
        }
    },
    "output": {
        "data": {
            "type": "integer",
            "description": "Random number between 0 and 10"
        },
        "example": 1
    },
}

In [3]:
# define skill client
def skill_client(broker_url, skill):
    sio = socketio.Client()
    
    @sio.on("taskRequest")
    def task(data):
        # the most important part of the skill - the actual task
        sio.emit('taskResults', {'id': data['id'], 'data': random.randint(0, 10)})
    
    @sio.on('connect')
    def on_connect():
        # register the skill when connected
        sio.emit('skillRegister', skill)
        
    # try to connect forever
    while True:
        try:
            logging.info("Connect to broker {}".format(broker_url))
            sio.connect(url)
            sio.wait()
        except:
            time.sleep(5)


In [4]:
# Start skill client
ctx = mp.get_context('fork')
client = ctx.Process(target=skill_client, args=(url, skill_definition))
client.start()
print("Skill client started")

Skill client started


In [ ]:
# to stop the skill client use the following
client.terminate()
client.join()
print("Skill client terminated")